In [24]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


Checkpoint function

In [25]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

Data load & pre-processing function

In [26]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Performance function

In [27]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    #Training 이 True일때는 dropout을 사용, False일떄는 dropout을 사용하지 않는다.
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Model function

In [28]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

Create model in class version

In [29]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() #RandomNormal에서 glorot_uniform으로 바꿔준다

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5)) #50프로만 제외하고 테스트하겠다는뜻


        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

Create model in function version

In [30]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

Define data & Hyper-parameter

In [31]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

Define model & optimizer & writer

In [32]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phrase

In [33]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer("C:/Users/johnk/logs/")
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        #checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.2975, train_loss: 2.26349783, train_accuracy: 0.3359, test_Accuracy: 0.1937
Epoch: [ 0] [    1/  468] time: 1.7812, train_loss: 2.36963201, train_accuracy: 0.2656, test_Accuracy: 0.2675
Epoch: [ 0] [    2/  468] time: 2.5073, train_loss: 2.29865432, train_accuracy: 0.2109, test_Accuracy: 0.2666
Epoch: [ 0] [    3/  468] time: 3.0598, train_loss: 2.29552031, train_accuracy: 0.3438, test_Accuracy: 0.3160
Epoch: [ 0] [    4/  468] time: 3.6712, train_loss: 2.23135543, train_accuracy: 0.3750, test_Accuracy: 0.4354
Epoch: [ 0] [    5/  468] time: 4.3354, train_loss: 2.28389096, train_accuracy: 0.4531, test_Accuracy: 0.5146
Epoch: [ 0] [    6/  468] time: 4.9138, train_loss: 2.27647710, train_accuracy: 0.5156, test_Accuracy: 0.5337
Epoch: [ 0] [    7/  468] time: 5.5013, train_loss: 2.18335295, train_accuracy: 0.4766, test_Accuracy: 0.5182
Epoch: [ 0] [    8/  468] time: 6.1326,

Epoch: [ 0] [   74/  468] time: 43.9189, train_loss: 0.65511137, train_accuracy: 0.8359, test_Accuracy: 0.8793
Epoch: [ 0] [   75/  468] time: 44.4564, train_loss: 0.65490437, train_accuracy: 0.8984, test_Accuracy: 0.8821
Epoch: [ 0] [   76/  468] time: 45.0763, train_loss: 0.50854146, train_accuracy: 0.9062, test_Accuracy: 0.8817
Epoch: [ 0] [   77/  468] time: 45.6343, train_loss: 0.59722835, train_accuracy: 0.8906, test_Accuracy: 0.8853
Epoch: [ 0] [   78/  468] time: 46.1649, train_loss: 0.44743681, train_accuracy: 0.9219, test_Accuracy: 0.8883
Epoch: [ 0] [   79/  468] time: 46.6686, train_loss: 0.72413957, train_accuracy: 0.9062, test_Accuracy: 0.8906
Epoch: [ 0] [   80/  468] time: 47.1780, train_loss: 0.48385948, train_accuracy: 0.9141, test_Accuracy: 0.8908
Epoch: [ 0] [   81/  468] time: 47.6537, train_loss: 0.50120729, train_accuracy: 0.9141, test_Accuracy: 0.8927
Epoch: [ 0] [   82/  468] time: 48.2950, train_loss: 0.58276141, train_accuracy: 0.8906, test_Accuracy: 0.8896
E

Epoch: [ 0] [  148/  468] time: 82.1166, train_loss: 0.34853297, train_accuracy: 0.9531, test_Accuracy: 0.9220
Epoch: [ 0] [  149/  468] time: 82.6816, train_loss: 0.39088506, train_accuracy: 0.9141, test_Accuracy: 0.9220
Epoch: [ 0] [  150/  468] time: 83.3448, train_loss: 0.32970816, train_accuracy: 0.9375, test_Accuracy: 0.9223
Epoch: [ 0] [  151/  468] time: 84.0509, train_loss: 0.25040668, train_accuracy: 0.9375, test_Accuracy: 0.9225
Epoch: [ 0] [  152/  468] time: 84.7141, train_loss: 0.66161680, train_accuracy: 0.8984, test_Accuracy: 0.9245
Epoch: [ 0] [  153/  468] time: 85.3597, train_loss: 0.40802372, train_accuracy: 0.9297, test_Accuracy: 0.9270
Epoch: [ 0] [  154/  468] time: 86.0040, train_loss: 0.39557433, train_accuracy: 0.9219, test_Accuracy: 0.9275
Epoch: [ 0] [  155/  468] time: 86.5645, train_loss: 0.33773315, train_accuracy: 0.9219, test_Accuracy: 0.9284
Epoch: [ 0] [  156/  468] time: 87.2014, train_loss: 0.39576820, train_accuracy: 0.9375, test_Accuracy: 0.9279
E

Epoch: [ 0] [  222/  468] time: 128.7848, train_loss: 0.33911204, train_accuracy: 0.9062, test_Accuracy: 0.9406
Epoch: [ 0] [  223/  468] time: 129.6505, train_loss: 0.36503237, train_accuracy: 0.9453, test_Accuracy: 0.9394
Epoch: [ 0] [  224/  468] time: 130.4723, train_loss: 0.39461046, train_accuracy: 0.9219, test_Accuracy: 0.9397
Epoch: [ 0] [  225/  468] time: 131.1465, train_loss: 0.43087339, train_accuracy: 0.9141, test_Accuracy: 0.9396
Epoch: [ 0] [  226/  468] time: 131.7649, train_loss: 0.35131949, train_accuracy: 0.9375, test_Accuracy: 0.9400
Epoch: [ 0] [  227/  468] time: 132.3583, train_loss: 0.41092649, train_accuracy: 0.9609, test_Accuracy: 0.9407
Epoch: [ 0] [  228/  468] time: 132.9148, train_loss: 0.27446848, train_accuracy: 0.9531, test_Accuracy: 0.9402
Epoch: [ 0] [  229/  468] time: 133.4803, train_loss: 0.26266673, train_accuracy: 0.9141, test_Accuracy: 0.9413
Epoch: [ 0] [  230/  468] time: 134.0049, train_loss: 0.29699326, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 172.7933, train_loss: 0.23250711, train_accuracy: 0.9531, test_Accuracy: 0.9453
Epoch: [ 0] [  297/  468] time: 173.3029, train_loss: 0.25779188, train_accuracy: 0.9453, test_Accuracy: 0.9456
Epoch: [ 0] [  298/  468] time: 173.8066, train_loss: 0.34910455, train_accuracy: 0.9062, test_Accuracy: 0.9442
Epoch: [ 0] [  299/  468] time: 174.2733, train_loss: 0.30298945, train_accuracy: 0.9453, test_Accuracy: 0.9432
Epoch: [ 0] [  300/  468] time: 174.7331, train_loss: 0.32422924, train_accuracy: 0.9609, test_Accuracy: 0.9434
Epoch: [ 0] [  301/  468] time: 175.2557, train_loss: 0.26364112, train_accuracy: 0.9844, test_Accuracy: 0.9418
Epoch: [ 0] [  302/  468] time: 175.8481, train_loss: 0.27933627, train_accuracy: 0.9219, test_Accuracy: 0.9420
Epoch: [ 0] [  303/  468] time: 176.5063, train_loss: 0.31581044, train_accuracy: 0.9219, test_Accuracy: 0.9417
Epoch: [ 0] [  304/  468] time: 177.1257, train_loss: 0.41204846, train_accuracy: 0.9219, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 221.1588, train_loss: 0.28313777, train_accuracy: 0.9531, test_Accuracy: 0.9505
Epoch: [ 0] [  371/  468] time: 222.0135, train_loss: 0.28498212, train_accuracy: 0.9688, test_Accuracy: 0.9512
Epoch: [ 0] [  372/  468] time: 223.1326, train_loss: 0.23922555, train_accuracy: 0.9609, test_Accuracy: 0.9514
Epoch: [ 0] [  373/  468] time: 223.8357, train_loss: 0.37579647, train_accuracy: 0.9219, test_Accuracy: 0.9514
Epoch: [ 0] [  374/  468] time: 224.4590, train_loss: 0.27462336, train_accuracy: 0.9609, test_Accuracy: 0.9506
Epoch: [ 0] [  375/  468] time: 225.2419, train_loss: 0.28955749, train_accuracy: 0.9531, test_Accuracy: 0.9501
Epoch: [ 0] [  376/  468] time: 225.9869, train_loss: 0.18604982, train_accuracy: 0.9609, test_Accuracy: 0.9489
Epoch: [ 0] [  377/  468] time: 226.6252, train_loss: 0.28788179, train_accuracy: 0.9531, test_Accuracy: 0.9468
Epoch: [ 0] [  378/  468] time: 227.3134, train_loss: 0.22110994, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 284.9740, train_loss: 0.28305155, train_accuracy: 0.9609, test_Accuracy: 0.9560
Epoch: [ 0] [  445/  468] time: 285.8726, train_loss: 0.24159104, train_accuracy: 0.9688, test_Accuracy: 0.9556
Epoch: [ 0] [  446/  468] time: 286.7603, train_loss: 0.18783551, train_accuracy: 0.9688, test_Accuracy: 0.9560
Epoch: [ 0] [  447/  468] time: 287.6050, train_loss: 0.28762713, train_accuracy: 0.9609, test_Accuracy: 0.9552
Epoch: [ 0] [  448/  468] time: 288.2313, train_loss: 0.38391641, train_accuracy: 0.9531, test_Accuracy: 0.9549
Epoch: [ 0] [  449/  468] time: 289.0023, train_loss: 0.27059942, train_accuracy: 0.9375, test_Accuracy: 0.9557
Epoch: [ 0] [  450/  468] time: 289.7532, train_loss: 0.30171967, train_accuracy: 0.9453, test_Accuracy: 0.9555
Epoch: [ 0] [  451/  468] time: 290.2798, train_loss: 0.16396919, train_accuracy: 0.9531, test_Accuracy: 0.9554
Epoch: [ 0] [  452/  468] time: 291.0189, train_loss: 0.15185997, train_accuracy: 0.9688, test_Accuracy:

In [ ]:
Test accuracy = 95.7% 